In [247]:
import pandas as pd
# Load the imputed median sale price data (from Zillow)
median_sale_price_df = pd.read_csv('sam-resources/Metro_median_sale_price_uc_sfrcondo_month_imputed.csv')
# Load the interpolated median sale price data (from Zillow)
median_sale_price_df_interpolated = pd.read_csv('sam-resources/Metro_median_sale_price_uc_sfrcondo_month_interpolated.csv')
# Load the census data (from census API)
census_data_df = pd.read_csv('sam-resources/census_data_2009-2022.csv')
# Load the median household income data (from census API)
median_household_income_df = pd.read_csv('sam-resources/median_income_2009-2022.csv')
# Load the imputed median list price data (from Zillow)
median_list_price_df = pd.read_csv('sam-resources/Median_list_Price_imputed.csv')
# Load the interpolated median list price data (from Zillow)
median_list_price_df_interpolated = pd.read_csv('sam-resources/Median_list_Price_interpolated.csv')
# Load the number of homes sold data
sales_total_df = pd.read_csv('sam-resources/ExistingHomeSales.csv')
# Load the mortgage interest rate data
mortgage_interest_rate_df = pd.read_csv('sam-resources/MORTGAGE30US.csv')


# List of all the census dataframes
census_df_list = [census_data_df, median_household_income_df]


In [248]:
median_household_income_df.head()

,City,Median Income,Year
0,"Marueño comunidad, Unknown",8438.0,2009
1,"Bayamón comunidad, Unknown",16900.0,2009
2,"Canóvanas zona urbana, Unknown",30857.0,2009
3,"Cabán comunidad, Unknown",9260.0,2009
4,"Oneonta city, AL",39113.0,2009


In [249]:
def calculate_housing_affordability_index(med_price, interest_rate, med_income):
    """
    Calculate the Housing Affordability Index (HAI)

    :param med_price: Median price of existing single-family home sale
    :param interest_rate: Interest rate (annual)
    :param med_income: Median family income
    :return: Housing Affordability Index
    """
    # Convert annual interest rate to monthly and calculate the monthly payment (PMT)
    monthly_interest_rate = interest_rate / 12 / 100
    pmt = med_price * 0.8 * (monthly_interest_rate) / (1 - (1 / ((1 + monthly_interest_rate)**360)))

    # Calculate Qualifying Income (QINC)
    qualifying_income = pmt * 4 * 12

    # Calculate Housing Affordability Index
    hai = (med_income / qualifying_income) * 100

    return hai

In [250]:
def merge_census_data(census_df_list):
    """
    Merge multiple census dataframes based on common columns.

    Parameters:
    census_df_list (list): A list of pandas DataFrames containing census data.

    Returns:
    pandas.DataFrame: A merged DataFrame containing all the census data.

    """
    all_census_df = census_df_list[0]
    for df in census_df_list[1:]:
        all_census_df = pd.merge(all_census_df, df, on=['City', 'Year', 'Median Income'], how='outer')
    return all_census_df

all_census_df = merge_census_data(census_df_list)

all_census_df.head()

,City,Total Population,Total Labor Force,Unemployed Labor Force,Median Income,Year
0,"Pine Flat CDP, CA",114.0,22.0,4.0,15724.0,2011
1,"Pine Grove CDP, CA",2573.0,1098.0,64.0,52917.0,2011
2,"Pine Mountain Lake CDP, CA",2695.0,942.0,100.0,54200.0,2011
3,"Piñon Hills CDP, CA",6130.0,2510.0,503.0,38140.0,2011
4,"Pioneer CDP, CA",1226.0,621.0,109.0,42917.0,2011


In [251]:
# Count nan values in each column of median_household_income_df
median_household_income_df.isna().sum()

City                0
Median Income    1170
Year                0
dtype: int64

# Data Cleaning

In [252]:
def clean_zillow_data(zdf, value_name='MedianSalePrice'):
    """
    Clean the Zillow data

    :param zdf: Zillow dataframe
    :return: Cleaned Zillow dataframe
    """
    # Rename the 'RegionName' column to 'Name'
    zdf = zdf.rename(columns={'RegionName': 'Name'})
    # Remove the state abbreviation from the values in the 'Name' column of zdf
    zdf['Name'] = zdf['Name'].str.split(',').str[0]
    # Rename the 'StateName' column in zdf to 'State'
    zdf = zdf.rename(columns={'StateName': 'State'})
    # Get a list of the column names in zdf that do not contain '-' or '/'
    id_vars = [col for col in zdf.columns if ('-' not in col) and ('/' not in col)]
    # Convert zdf from wide to long format
    zdf = pd.melt(zdf, id_vars = id_vars, var_name='Date', value_name = value_name)
    cols = ['Date', 'Name', 'State', value_name]
    # Keep only the columns specified in cols
    zdf = zdf[cols]
    # Add a column to zdf that contains the year extracted from the 'Date' column
    if '-' in zdf['Date'][0]:
        zdf['Year'] = zdf['Date'].str.split('-').str[0]
    elif '/' in zdf['Date'][0]:
        zdf['Year'] = zdf['Date'].str.split('/').str[-1]
    # Remove the leading and trailing whitespace from the values in the 'State' Column of zdf
    zdf['State'] = zdf['State'].str.strip()
    # Remove all leading and trailing whitespace from the values in the 'Name' column of zdf
    zdf['Name'] = zdf['Name'].str.strip()
    # Convert 'Date' to datetime in zdf
    zdf['Date'] = pd.to_datetime(zdf['Date'])
    # Convert 'Year' to int64 in zdf
    zdf['Year'] = zdf['Year'].astype('int64')
    # Return the cleaned zillow dataframe
    return zdf


median_sale_price_df = clean_zillow_data(median_sale_price_df, 'MedianSalePrice')
median_list_price_df = clean_zillow_data(median_list_price_df, 'MedianListPrice')
sales_total_df = clean_zillow_data(sales_total_df, 'SalesTotal')
median_list_price_df_interpolated = clean_zillow_data(median_list_price_df_interpolated, 'MedianListPrice')
median_sale_price_df_interpolated = clean_zillow_data(median_sale_price_df_interpolated, 'MedianSalePrice')
median_sale_price_df.head()

,Date,Name,State,MedianSalePrice,Year
0,2008-02-29,United States,NaN,174000.0,2008
1,2008-02-29,New York,NY,400000.0,2008
2,2008-02-29,Los Angeles,CA,470000.0,2008
3,2008-02-29,Chicago,IL,224000.0,2008
4,2008-02-29,Dallas,TX,138000.0,2008


In [253]:
# List of all the interpolated zillow dataframes
zillow_df_interpolated_list = [median_sale_price_df_interpolated, median_list_price_df_interpolated, sales_total_df]

In [254]:
# List of all the zillow dataframes
zillow_df_list = [median_sale_price_df, median_list_price_df, sales_total_df]

In [255]:
def merge_zillow_data(zillow_df_list):
    """
    Merge multiple Zillow dataframes based on common columns.

    Parameters:
    zillow_df_list (list): A list of Zillow dataframes to be merged.

    Returns:
    pandas.DataFrame: The merged dataframe containing all the data from the input dataframes.
    """
    all_zillow_df = zillow_df_list[0]
    for df in zillow_df_list[1:]:
        all_zillow_df = pd.merge(all_zillow_df, df, on=['Date', 'Name', 'State', 'Year'], how='outer')
    return all_zillow_df
all_zillow_df = merge_zillow_data(zillow_df_list)
all_zillow_df.head()

,Date,Name,State,MedianSalePrice,Year,MedianListPrice,SalesTotal
0,2008-02-29,United States,NaN,174000.0,2008,NaN,205206.0
1,2008-02-29,New York,NY,400000.0,2008,NaN,8591.0
2,2008-02-29,Los Angeles,CA,470000.0,2008,NaN,4159.0
3,2008-02-29,Chicago,IL,224000.0,2008,NaN,5933.0
4,2008-02-29,Dallas,TX,138000.0,2008,NaN,5058.0


In [256]:
all_zillow_df_interpolated = merge_zillow_data(zillow_df_interpolated_list)
all_zillow_df_interpolated.head()

,Date,Name,State,MedianSalePrice,Year,MedianListPrice,SalesTotal
0,2008-02-29,United States,NaN,174000.0,2008,NaN,205206.0
1,2008-02-29,New York,NY,400000.0,2008,NaN,8591.0
2,2008-02-29,Los Angeles,CA,470000.0,2008,NaN,4159.0
3,2008-02-29,Chicago,IL,224000.0,2008,NaN,5933.0
4,2008-02-29,Dallas,TX,138000.0,2008,NaN,5058.0


In [257]:
def clean_census_data(cdf, zdf):
    """
    Clean the Census data

    :param cdf: Census dataframe
    :param zdf: Zillow dataframe
    :return: Cleaned Census dataframe
    """
    # Add a column to cdf that contains the state abbreviation extracted from the 'City' column
    cdf['State'] = cdf['City'].str.split(',').str[1]
    # Rename the 'City' column to 'Name'
    cdf = cdf.rename(columns={'City': 'Name'})
    # Remove the state abbreviation from the values in the 'Name' column of cdf
    cdf['Name'] = cdf['Name'].str.split(',').str[0]
    # Remove all rows in cdf that do not contain 'city' in the 'Name' column
    cdf = cdf[cdf['Name'].str.contains('city')]
    # Remove 'city' from the values in the 'Name' column of cdf
    cdf['Name'] = cdf['Name'].str.split('city').str[0]
    # Remove all leading and trailing whitespace from the values in the 'Name' column of cdf
    cdf['Name'] = cdf['Name'].str.strip()
    # Remove the leading and trailing whitespace from the values in the 'State' column of cdf
    cdf['State'] = cdf['State'].str.strip()
    # Convert 'Year' to int64 in cdf
    cdf['Year'] = cdf['Year'].astype('int64')
    # Filter cdf to only include rows where the 'Name' column is in the 'Name' column of zdf
    cdf = cdf[cdf['Name'].isin(zdf['Name'])]
    # Return the cleaned census dataframe
    return cdf



all_census_df = clean_census_data(all_census_df, all_zillow_df)
all_census_df.head()

,Name,Total Population,Total Labor Force,Unemployed Labor Force,Median Income,Year,State
233,Barnstable Town,45486.0,24276.0,1662.0,62191.0,2011,MA
235,Little Rock,461.0,241.0,11.0,41250.0,2011,IA
252,Hartford,669.0,319.0,12.0,49453.0,2011,IA
290,Hammond,19926.0,10069.0,893.0,33544.0,2011,LA
297,Baton Rouge,229169.0,118979.0,10975.0,37381.0,2011,LA


In [258]:
mortgage_interest_rate_df = mortgage_interest_rate_df.rename(columns={'DATE': 'Date'})
# Convert 'Date' to datetime in mortgage_interest_rate_df
mortgage_interest_rate_df['Date'] = pd.to_datetime(mortgage_interest_rate_df['Date'])

In [259]:
def final_merge(all_zillow_df, all_census_df, mortgage_interest_rate_df):
    df = pd.merge(all_zillow_df, all_census_df, on=['Name', 'State', 'Year'], how='outer')
    # Merge df and mortgage_interest_rate_df on the 'Date' column
    df = pd.merge(df, mortgage_interest_rate_df, on='Date', how = 'outer')
    df['Unemployment Rate'] = (df['Unemployed Labor Force'] / df['Total Labor Force']) * 100 # Calculate the unemployment rate
    # Apply the calculate_housing_affordability_index function to the df DataFrame
    df['HAI'] = calculate_housing_affordability_index(df['MedianSalePrice'], df['MORTGAGE30US'], df['Median Income'])
    # Remove the 'Total Labor Force' and the 'Unemployed Labor Force' columns from df
    df = df.drop(columns=['Total Labor Force', 'Unemployed Labor Force'])
    # Rename the 'Name' column to 'CityName'
    df = df.rename(columns={'Name': 'CityName'})
    # Rename the 'State' column to 'StateName' 
    df = df.rename(columns={'State': 'StateName'})
    return df

df = final_merge(all_zillow_df, all_census_df, mortgage_interest_rate_df)
df_interpolated = final_merge(all_zillow_df_interpolated, all_census_df, mortgage_interest_rate_df)
df.head()


,Date,CityName,StateName,MedianSalePrice,Year,MedianListPrice,SalesTotal,Total Population,Median Income,MORTGAGE30US,Unemployment Rate,HAI
0,2008-02-29,United States,NaN,174000.0,2008.0,NaN,205206.0,NaN,NaN,NaN,NaN,NaN
1,2008-02-29,New York,NY,400000.0,2008.0,NaN,8591.0,NaN,NaN,NaN,NaN,NaN
2,2008-02-29,Los Angeles,CA,470000.0,2008.0,NaN,4159.0,NaN,NaN,NaN,NaN,NaN
3,2008-02-29,Chicago,IL,224000.0,2008.0,NaN,5933.0,NaN,NaN,NaN,NaN,NaN
4,2008-02-29,Dallas,TX,138000.0,2008.0,NaN,5058.0,NaN,NaN,NaN,NaN,NaN


In [260]:
df_interpolated.head()

,Date,CityName,StateName,MedianSalePrice,Year,MedianListPrice,SalesTotal,Total Population,Median Income,MORTGAGE30US,Unemployment Rate,HAI
0,2008-02-29,United States,NaN,174000.0,2008.0,NaN,205206.0,NaN,NaN,NaN,NaN,NaN
1,2008-02-29,New York,NY,400000.0,2008.0,NaN,8591.0,NaN,NaN,NaN,NaN,NaN
2,2008-02-29,Los Angeles,CA,470000.0,2008.0,NaN,4159.0,NaN,NaN,NaN,NaN,NaN
3,2008-02-29,Chicago,IL,224000.0,2008.0,NaN,5933.0,NaN,NaN,NaN,NaN,NaN
4,2008-02-29,Dallas,TX,138000.0,2008.0,NaN,5058.0,NaN,NaN,NaN,NaN,NaN


In [268]:
len(df_interpolated)

142975

In [269]:
len(df)

142975

In [261]:
df.to_csv('sam-resources/data.csv', index=False)

In [262]:
df_interpolated.to_csv('sam-resources/data_interpolated.csv', index=False)

In [263]:
df[['Date', 'CityName', 'SalesTotal']] [df['CityName']=='New York']

,Date,CityName,SalesTotal
1,2008-02-29,New York,8591.0
676,2008-03-31,New York,8967.0
1351,2008-04-30,New York,9977.0
2026,2008-05-31,New York,10232.0
2701,2008-06-30,New York,10854.0
...,...,...,...
124876,2023-07-31,New York,13076.0
125551,2023-08-31,New York,15184.0
126226,2023-09-30,New York,12678.0
126901,2023-10-31,New York,12449.0


In [264]:
df[['Date', 'CityName', 'MORTGAGE30US']] [df['CityName']=='New York'].dropna(subset=['MORTGAGE30US'])


,Date,CityName,MORTGAGE30US
3376,2008-07-31,New York,6.52
6751,2008-12-31,New York,5.10
9451,2009-04-30,New York,4.78
14851,2009-12-31,New York,5.14
20926,2010-09-30,New York,4.32
24976,2011-03-31,New York,4.86
27001,2011-06-30,New York,4.51
34426,2012-05-31,New York,3.75
39826,2013-01-31,New York,3.53
40501,2013-02-28,New York,3.51


In [265]:
df[['Date', 'CityName', 'Median Income']] [df['CityName']=='New York'].dropna(subset=['Median Income'])


,Date,CityName,Median Income
7426,2009-01-31,New York,50173.0
8101,2009-02-28,New York,50173.0
8776,2009-03-31,New York,50173.0
9451,2009-04-30,New York,50173.0
10126,2009-05-31,New York,50173.0
...,...,...,...
117451,2022-08-31,New York,76607.0
118126,2022-09-30,New York,76607.0
118801,2022-10-31,New York,76607.0
119476,2022-11-30,New York,76607.0


In [266]:
df.head()

,Date,CityName,StateName,MedianSalePrice,Year,MedianListPrice,SalesTotal,Total Population,Median Income,MORTGAGE30US,Unemployment Rate,HAI
0,2008-02-29,United States,NaN,174000.0,2008.0,NaN,205206.0,NaN,NaN,NaN,NaN,NaN
1,2008-02-29,New York,NY,400000.0,2008.0,NaN,8591.0,NaN,NaN,NaN,NaN,NaN
2,2008-02-29,Los Angeles,CA,470000.0,2008.0,NaN,4159.0,NaN,NaN,NaN,NaN,NaN
3,2008-02-29,Chicago,IL,224000.0,2008.0,NaN,5933.0,NaN,NaN,NaN,NaN,NaN
4,2008-02-29,Dallas,TX,138000.0,2008.0,NaN,5058.0,NaN,NaN,NaN,NaN,NaN


In [267]:
df.dropna(subset=['HAI', 'SalesTotal'])

,Date,CityName,StateName,MedianSalePrice,Year,MedianListPrice,SalesTotal,Total Population,Median Income,MORTGAGE30US,Unemployment Rate,HAI
9451,2009-04-30,New York,NY,339000.0,2009.0,NaN,6974.0,NaN,50173.0,4.78,NaN,73.630547
9452,2009-04-30,Los Angeles,CA,320000.0,2009.0,NaN,8168.0,NaN,48570.0,4.78,NaN,75.510228
9453,2009-04-30,Chicago,IL,192000.0,2009.0,NaN,5582.0,NaN,46781.0,4.78,NaN,121.214878
9454,2009-04-30,Dallas,TX,140000.0,2009.0,NaN,4811.0,NaN,41266.0,4.78,NaN,146.639845
9455,2009-04-30,Houston,TX,145000.0,2009.0,NaN,3841.0,NaN,42797.0,4.78,NaN,146.836147
...,...,...,...,...,...,...,...,...,...,...,...,...
116191,2022-06-30,Provo,UT,513000.0,2022.0,568333.0,982.0,114400.0,57943.0,5.70,4.404018,50.678606
116192,2022-06-30,Toledo,OH,152000.0,2022.0,171467.0,908.0,269962.0,45405.0,5.70,8.180164,134.029730
116193,2022-06-30,Wichita,KS,230000.0,2022.0,211667.0,524.0,395951.0,60712.0,5.70,5.655765,118.437098
116194,2022-06-30,Durham,NC,400000.0,2022.0,409967.0,1179.0,284094.0,74710.0,5.70,4.283409,83.803045
